# PHASE I: Régression Linéaire Avancée et Analyse des Compromis

## Tache 1.1 - Charger et Préparer

In [1]:
# 1. Import des bibliothèques nécessaires
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#2. Chargement du dataset Diabetes
X, y = load_diabetes(return_X_y=True)

#3. Division en train(80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#4. Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Tache 1.2 - Régression OLS

In [2]:
# 1. Créer et entraîner un modèle OLS (Ordinary Least Squares)
ols_reg = LinearRegression()
ols_reg.fit(X_train_scaled, y_train)

#2. Prédictions sur le test set
y_pred_ols = ols_reg.predict(X_test_scaled)

#3. Calcul des metriques
rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))
r2_ols = r2_score(y_test, y_pred_ols)

#4. Affichage des resultats R² % variance expliquée (0 à 1, plus haut = mieux)
print(f"OLS: RMSE = {rmse_ols:.2f}, R² = {r2_ols:.2f}")
print("Coefficients OLS:", ols_reg.coef_)

OLS: RMSE = 54.52, R² = 0.44
Coefficients OLS: [  1.75375799 -11.51180908  25.60712144  16.82887167 -44.44885564
  24.64095356   7.67697768  13.1387839   35.16119521   2.35136365]


## Tache 1.3 - THÉORIE

### 1. Biais (Bias)
Le biais est une erreur due aux hypothèses simplificatrices du modèle
   - Modèle TROP SIMPLE → sous-apprentissage (underfitting)
   - Exemple: régression linéaire sur données non-linéaires

### 2.  VARIANCE:
La variance est une erreur due a une forte sensibilité du modèle aux variations dans les données d'entraînement
   - Modèle TROP COMPLEXE → sur-apprentissage (overfitting)
   - Exemple: polynôme degré 50 sur 100 points

### 3. RELATION:
   Erreur totale = Bias² + Variance + Bruit irréductible
   
   Trade-off: Réduire l'un augmente l'autre
   - ↓ Bias → ↑ Complexité → ↑ Variance
   - ↓ Variance → ↓ Complexité → ↑ Bias

### 4. COMMENT LA RÉGULARISATION MODIFIE CE TRADE-OFF:
   
   Sans régularisation (OLS):
   Loss = MSE = Σ(y - ŷ)²
   → Minimise erreur training → Risque overfitting (haute variance)
   
   Avec régularisation:
   Loss = MSE + λ × Pénalité(coefficients)
   
   Ridge (L2): Loss = MSE + λ × Σwᵢ²
   LASSO (L1): Loss = MSE + λ × Σ|wᵢ|
   
   EFFET DE λ (hyperparamètre):
   - λ = 0: Pas de régularisation → OLS standard (haute variance possible)
   - λ petit: Légère contrainte → réduit variance un peu
   - λ grand: Forte contrainte → réduit variance beaucoup mais ↑ bias
   - λ → ∞: Tous coefficients → 0 → modèle constant (bias max)

### 5. PRÉVENTION DE L'OVERFITTING:
   
   La régularisation ajoute une CONTRAINTE sur la magnitude des coefficients:
   - Empêche les coefficients d'exploser
   - Force le modèle à être plus SIMPLE
   - Réduit la sensibilité au bruit dans les données
   - Améliore la GÉNÉRALISATION sur nouvelles données
   
   Résultat: ↑ Bias légèrement, mais ↓↓ Variance significativement
   → Erreur test globale diminue (meilleure généralisation)


## Tache 1.4 - Ridge Regression (L2)

In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# 1. Entrainement du modele Ridge avec alpha=1.0
ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X_train_scaled, y_train)

# 2. Evaluation par validation croisée
scores_ridge = cross_val_score(
    ridge_reg, X_train_scaled, y_train,
    scoring='neg_root_mean_squared_error', cv=5
)
rmse_cv_ridge = -scores_ridge.mean()


# 3. Afficher résultats
print(f"Ridge (alpha=1.0): RMSE CV={rmse_cv_ridge:.2f}")
print("Coefficients Ridge:", ridge_reg.coef_)

Ridge (alpha=1.0): RMSE CV=55.92
Coefficients Ridge: [  1.80734179 -11.44818951  25.73269892  16.73429974 -34.67195409
  17.05307485   3.36991411  11.76426044  31.3783838    2.45813922]


## POURQUOI RIDGE RÉDUIT MAIS NE MET PAS À ZÉRO

1. PÉNALITÉ L2 vs L1:
   
   - Ridge (L2): Pénalité = λ × Σwᵢ²
   - LASSO (L1): Pénalité = λ × Σ|wᵢ|

---
2. DÉRIVÉE - C'EST LA CLÉ:
   
   Ridge: ∂(wᵢ²)/∂wᵢ = 2wᵢ
   → Gradient proportionnel au coefficient
   → Quand wᵢ → 0, gradient → 0 aussi
   → Réduction ASYMPTOTIQUE vers 0 mais jamais 0 exactement
   
   LASSO: ∂|wᵢ|/∂wᵢ = sign(wᵢ) = {+1 si wᵢ>0, -1 si wᵢ<0}
   → Gradient CONSTANT quelle que soit la valeur de wᵢ
   → Pousse directement à 0 (seuillage dur)
---
3. GÉOMÉTRIE - FORME DES CONTRAINTES:
   
   Ridge: Contrainte = boule (cercle en 2D)
   → Bords lisses et courbes
   → Solution optimale peut être n'importe où sur le bord
   → Probabilité faible de toucher les axes (w=0)
   
   LASSO: Contrainte = losange (diamant en 2D)
   → COINS sur les axes
   → Solution optimale tend vers les coins
   → Probabilité ÉLEVÉE que w = 0 exactement
---
4. COMPORTEMENT PRATIQUE:
   
   Ridge avec λ croissant:
   w = [100, 50, 10] → [50, 25, 5] → [10, 5, 1] → [0.01, 0.005, 0.001]
   → JAMAIS exactement 0, mais très proche
   
   LASSO avec λ croissant:
   w = [100, 50, 10] → [80, 30, 0] → [50, 0, 0] → [0, 0, 0]
   → Met à 0 progressivement (sélection de features)
---
5. AVANTAGE RIDGE:
   - Garde toutes les features (utile si toutes importantes)
   - Stable numériquement
   - Solution unique même si p > n
   
   AVANTAGE LASSO:
   - Sélection automatique de features
   - Modèle sparse et interprétable
   - Réduit la dimensionnalité


## 1.3 Sparsity et Optimisation des Hyperparamètres

## TASK 1.5 - LASSO et Concept de Sparsity


In [7]:
from sklearn.linear_model import Lasso

# 1. Entraîner LASSO avec alpha initial
lasso_reg = Lasso(alpha=0.1, max_iter=10000)
lasso_reg.fit(X_train_scaled, y_train)

# 2. Afficher coefficients
print("Coefficients LASSO:", lasso_reg.coef_)
print(f"Nombre de coefficients = 0: {np.sum(lasso_reg.coef_ == 0)}")

Coefficients LASSO: [  1.73045056 -11.31635911  25.82462699  16.64425156 -29.35841191
  13.27584411   0.5479479   10.23616805  29.63282611   2.39347521]
Nombre de coefficients = 0: 0


## Sparsity (Parcimonie)

### Définition
La sparsity désigne un modèle dont **beaucoup de coefficients sont exactement nuls**.

\[
w = (w_1,\dots,w_p), \quad \text{avec } w_j = 0 \text{ pour beaucoup de } j
\]

---

### Intérêt (interprétabilité)
- Moins de variables actives
- Modèle plus lisible
- Identification claire des features utiles

---

### Pourquoi L1 crée des zéros et pas L2 ?

### Pénalités
- **L1 (LASSO)** :  
\[
\lambda \sum_j |w_j|
\]

- **L2 (Ridge)** :  
\[
\lambda \sum_j w_j^2
\]

---

### Différentiabilité
- **L2** : dérivable partout → coefficients réduits mais jamais nuls
- **L1** : non différentiable en 0 → solution possible exactement en 0

---

### Géométrie
- **L2** : contrainte circulaire → contact rarement sur les axes
- **L1** : contrainte en losange → coins sur les axes → coefficients nuls

---

### Soft Thresholding (LASSO)
\[
\hat w = \text{sign}(z)\max(|z| - \lambda, 0)
\]

- Si \(|z| \le \lambda\) → \(w = 0\)

---

## Avantages du LASSO
- Sélection automatique de variables
- Réduction de dimension
- Efficace quand \(p > n\)


##  Tache 1.6 - Optimisation avec Cross-Validation

In [8]:
from sklearn.linear_model import RidgeCV, LassoCV

# 1. Ridge Tuning - teste 100 valeurs d'alpha
ridge_cv = RidgeCV(alphas=np.logspace(-3, 2, 100), cv=10)
ridge_cv.fit(X_train_scaled, y_train)
alpha_ridge_optimal = ridge_cv.alpha_

print(f"Alpha optimal pour Ridge: {alpha_ridge_optimal:.4f}")

# 2. LASSO Tuning - teste 100 valeurs d'alpha
lasso_cv = LassoCV(alphas=np.logspace(-3, 0, 100), cv=10, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)
alpha_lasso_optimal = lasso_cv.alpha_

print(f"Alpha optimal pour LASSO: {alpha_lasso_optimal:.4f}")

# 3. Modèle LASSO final avec alpha optimal
lasso_final = Lasso(alpha=alpha_lasso_optimal, max_iter=10000)
lasso_final.fit(X_train_scaled, y_train)

# 4. Analyse de la Sparsity
print("\nCoefficients LASSO finaux:")
for i, coef in enumerate(lasso_final.coef_):
    status = "→ ZÉRO" if coef == 0 else ""
    print(f"  Feature {i}: {coef:.4f} {status}")

zero_count = np.sum(lasso_final.coef_ == 0)
print(f"\nFeatures éliminées: {zero_count}/{len(lasso_final.coef_)}")

# 5. Évaluation finale des 3 modèles
y_pred_ridge = ridge_cv.predict(X_test_scaled)
y_pred_lasso = lasso_final.predict(X_test_scaled)

rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_ridge = r2_score(y_test, y_pred_ridge)
r2_lasso = r2_score(y_test, y_pred_lasso)

# 6. Tableau comparatif
print("\n📊 COMPARAISON FINALE:")
print(f"{'Modèle':<10} {'RMSE':>8} {'R²':>8} {'Features':>10}")
print("-" * 40)
print(f"{'OLS':<10} {rmse_ols:>8.2f} {r2_ols:>8.4f} {10:>10}")
print(f"{'Ridge':<10} {rmse_ridge:>8.2f} {r2_ridge:>8.4f} {10:>10}")
print(f"{'LASSO':<10} {rmse_lasso:>8.2f} {r2_lasso:>8.4f} {10-zero_count:>10}")

Alpha optimal pour Ridge: 55.9081
Alpha optimal pour LASSO: 0.1417

Coefficients LASSO finaux:
  Feature 0: 1.6966 
  Feature 1: -11.2072 
  Feature 2: 25.9031 
  Feature 3: 16.5961 
  Feature 4: -27.1653 
  Feature 5: 11.3606 
  Feature 6: -0.0000 → ZÉRO
  Feature 7: 10.3508 
  Feature 8: 28.7237 
  Feature 9: 2.3938 

Features éliminées: 1/10

📊 COMPARAISON FINALE:
Modèle         RMSE       R²   Features
----------------------------------------
OLS           54.52   0.4389         10
Ridge         53.88   0.4521         10
LASSO         54.23   0.4449          9


### CE QU'ON OBSERVE:

- Ridge garde toutes les features mais réduit leurs coefficients
- LASSO élimine certaines features (coefficients = 0)
- Comparaison des performances (RMSE, R²)

# PHASE II: MLOps - Du Modèle à la Production

## 2.1 Packaging du Modèle et Gestion des Dépendances


In [9]:
import joblib

# 1. Sauvegarder le meilleur modèle (LASSO) et le scaler
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(lasso_final, 'lasso_model.pkl')

print("✓ Scaler sauvegardé: scaler.pkl")
print("✓ Modèle LASSO sauvegardé: lasso_model.pkl")

# 2. Fonction de prédiction en production
def predict_new_data(raw_data, model_path='lasso_model.pkl', 
                     scaler_path='scaler.pkl'):

    try:
        # Charger les objets sauvegardés
        loaded_scaler = joblib.load(scaler_path)
        loaded_model = joblib.load(model_path)
    except FileNotFoundError:
        print("Erreur: Fichiers introuvables")
        return None
    
    # Étape 1: Transformer avec le MÊME scaler (CRUCIAL!)
    scaled_data = loaded_scaler.transform([raw_data])
    
    # Étape 2: Prédiction
    prediction = loaded_model.predict(scaled_data)[0]
    
    return prediction

# 3. Test de la fonction
example_input = X_test[0]
predicted = predict_new_data(example_input)
actual = y_test[0]

print(f"\n[Test Prédiction]")
print(f"  Prédite: {predicted:.2f}")
print(f"  Réelle: {actual:.2f}")
print(f"  Erreur: {abs(predicted - actual):.2f}")

✓ Scaler sauvegardé: scaler.pkl
✓ Modèle LASSO sauvegardé: lasso_model.pkl

[Test Prédiction]
  Prédite: 150.95
  Réelle: 219.00
  Erreur: 68.05


## Pourquoi sauvegarder le scaler ?

### 1. Cohérence des transformations
Un modèle apprend **sur des données transformées**, pas sur les données brutes.  
Si le scaler utilisé en production n’est pas **exactement** celui du training, alors les entrées du modèle ne sont plus dans le même espace.  
Même modèle, mêmes features, **mais monde mathématique différent** → prédictions incohérentes.

---

### 2. Exemple concret
**Training**
- Feature : BMI  
- Moyenne = 25  
- Écart-type = 5  

Standardisation :
\[
z = \frac{BMI - 25}{5}
\]

**Production**
- Nouvelle donnée : BMI = 30  

Avec le bon scaler :
\[
z = \frac{30 - 25}{5} = 1
\]

Sans le scaler sauvegardé (ex: moyenne=27, std=3 recalculés) :
\[
z = \frac{30 - 27}{3} = 1
\]

Même valeur brute, **représentation différente**, donc le modèle voit **un autre individu**.

---

### 3. Risques
- Prédictions fausses ou instables  
- Dégradation silencieuse des performances  
- Impossibilité de reproduire les résultats  
- Modèle inutilisable en production  
- Debug infernal sans cause évidente  

En clair : le modèle fonctionne, mais raconte n’importe quoi.

---

### 4. Règle d’or (MLOps)
**Scaler + modèle = un seul artefact.**  
On **fit** le scaler sur le training, on **le sauvegarde**, et on le **réutilise tel quel** en production.  
Jamais de refit sur les données de production. Jamais.


## 2.2 MLflow pour la Gestion du Cycle de Vie

### Tache 2.3 - Tracking avec MLflow

In [10]:
import mlflow

# 1. Configurer l'expérience
mlflow.set_experiment("TP_Advanced_Regression")

def log_model_results(model_name, params, metrics, model_object):
    """
    Log les résultats d'un modèle dans MLflow.
    
    Args:
        model_name: Nom du run (ex: "OLS", "Ridge", "LASSO")
        params: Dictionnaire des hyperparamètres
        metrics: Dictionnaire des métriques (RMSE, R²)
        model_object: Objet modèle sklearn
    """
    with mlflow.start_run(run_name=model_name):
        # Log hyperparamètres
        mlflow.log_params(params)
        
        # Log métriques
        mlflow.log_metrics(metrics)
        
        # Log modèle comme artifact
        mlflow.sklearn.log_model(model_object, name="model")
        
        print(f"✓ Run '{model_name}' enregistré dans MLflow")

# 2. Logger OLS
ols_params = {"model_type": "OLS", "regularization": "none"}
ols_metrics = {"test_rmse": rmse_ols, "test_r2": r2_ols}
log_model_results("OLS_Baseline", ols_params, ols_metrics, ols_reg)

# 3. Logger Ridge
ridge_params = {
    "alpha": alpha_ridge_optimal,
    "solver": "auto",
    "penalty": "l2"
}
ridge_metrics = {"test_rmse": rmse_ridge, "test_r2": r2_ridge}
log_model_results("Ridge_Optimal", ridge_params, ridge_metrics, ridge_cv)

# 4. Logger LASSO
lasso_params = {
    "alpha": alpha_lasso_optimal,
    "solver": "coordinate_descent",
    "penalty": "l1"
}
lasso_metrics = {
    "test_rmse": rmse_lasso,
    "test_r2": r2_lasso,
    "num_zero_coefs": zero_count,
    "sparsity_ratio": zero_count / len(lasso_final.coef_)
}
log_model_results("LASSO_Optimal", lasso_params, lasso_metrics, lasso_final)

print("\n✓ Tous les modèles enregistrés dans MLflow")
print("Pour visualiser: mlflow ui")
print("Puis ouvrir: http://localhost:5000")

2026/01/03 15:13:32 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/03 15:13:32 INFO mlflow.store.db.utils: Updating database tables
2026/01/03 15:13:32 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/03 15:13:32 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2026/01/03 15:13:33 INFO alembic.runtime.migration: Running 

✓ Run 'OLS_Baseline' enregistré dans MLflow
✓ Run 'Ridge_Optimal' enregistré dans MLflow
✓ Run 'LASSO_Optimal' enregistré dans MLflow

✓ Tous les modèles enregistrés dans MLflow
Pour visualiser: mlflow ui
Puis ouvrir: http://localhost:5000


## Explication du Dockerfile

### FROM
- **Rôle** : Définit l’image de base sur laquelle le conteneur est construit.  
  Elle fournit l’OS minimal, le runtime et les outils de base.
- **Pourquoi `python:3.9-slim` ?**  
  - Python déjà installé  
  - Image légère (moins de surface d’attaque, build plus rapide)  
  - Suffisant pour exécuter une app Python sans dépendances inutiles  

---

### WORKDIR
- **Rôle** : Définit le répertoire de travail à l’intérieur du conteneur.  
  Toutes les commandes suivantes (`COPY`, `RUN`, `CMD`) s’exécutent depuis ce dossier.  
  Évite les chemins absolus et le désordre.

---

### COPY
- **Rôle** : Copie des fichiers du système hôte vers le système de fichiers du conteneur.
- **Différence entre `COPY requirements.txt` et `COPY app.py` ?**  
  - `requirements.txt` : utilisé pour installer les dépendances  
  - `app.py` : contient le code applicatif  
  Séparer les deux permet à Docker de **mettre en cache l’installation des dépendances** si le code change mais pas les libs.

---

### RUN
- **Rôle** : Exécute une commande **au moment du build** de l’image.  
  Typiquement utilisé pour installer des dépendances ou configurer l’environnement.
- **Quand est exécutée ?**  
  Une seule fois, lors du `docker build`.  
  Le résultat est figé dans l’image finale.

---

### EXPOSE
- **Rôle** : Documente le port sur lequel l’application écoute à l’intérieur du conteneur.
- **Est-ce que ça ouvre vraiment le port ?**  
  Non.  
  Ça n’ouvre rien du tout.  
  Le port est réellement exposé uniquement avec `-p` ou `--publish` au `docker run`.

---

### ENV
- **Rôle** : Définit des variables d’environnement disponibles dans le conteneur.  
  Utilisées pour la configuration (mode debug, ports, chemins, clés, etc.).  
  Évite le hard-coding dans le code.

---

### CMD
- **Rôle** : Définit la commande **par défaut** exécutée au démarrage du conteneur.
- **Différence entre `CMD` et `RUN` ?**  
  - `RUN` → exécuté au build, une seule fois  
  - `CMD` → exécuté au runtime, à chaque lancement du conteneur  

Sans `CMD`, ton conteneur démarre… et s’arrête immédiatement. Classe.


## Synthèse MLOps: Git + MLflow + Docker

### 1. GIT – Version Control
**Rôle:**
- Gestion des versions du code
- Traçabilité des changements
- Collaboration propre et reproductible

**Pour le ML:**
- Versionnement du code d’entraînement
- Suivi des features, scripts, configs
- Lien clair entre code, données et modèle entraîné

**CI/CD:**
- `git push` déclenche automatiquement les pipelines (tests, training, build, deploy)
- Base de toute automatisation fiable

---

### 2. MLFLOW – Lifecycle Management

**4 Composantes:**
- **Tracking** : log des paramètres, métriques, artefacts → reproductibilité
- **Projects** : standardisation des runs (environnement, entry point)
- **Models** : format unifié pour charger/servir les modèles
- **Registry** : versioning, staging, production, rollback

**CI/CD:**
- Enregistrement automatique des modèles après training
- Promotion contrôlée (Staging → Production)
- Intégration directe dans les pipelines de déploiement

---

### 3. DOCKER – Containerisation

**Rôle:**
- Emballer code + dépendances + environnement
- Garantir “ça marche chez moi = ça marche partout”

**Avantages ML:**
- Environnement identique training / inference
- Isolation des dépendances (librairies, CUDA, versions)
- Déploiement reproductible

**CI/CD:**
- Build d’images automatisé
- Déploiement rapide sur serveur, cloud ou Kubernetes
- Rollback simple par version d’image

---

### 4. PIPELINE CI/CD COMPLET

**Workflow typique:**
1. **Data Scientist** : `git push`
2. **CI** :
   - Tests du code
   - Entraînement du modèle
   - Logging MLflow (metrics, modèle)
   - Build image Docker
3. **CD** :
   - Récupération du modèle validé (Registry)
   - Déploiement automatique en production
4. **Monitoring** :
   - Surveillance des données, performances, dérives
   - Alertes et retraining si nécessaire

---

### 5. EXEMPLE CONCRET

**Sans MLOps:**
- Modèle entraîné localement
- Résultats non reproductibles
- Déploiement manuel
- Bugs et dérives non détectés

**Avec MLOps:**
- Code versionné (Git)
- Modèle traçable et versionné (MLflow)
- Déploiement stable et reproductible (Docker)
- Monitoring continu et rollback possible

---

## Monitoring Post-Production

### 1. DATA DRIFT (Dérive des Données)

**Définition:**
Changement de la distribution des données d’entrée entre le training et la production.

**Causes:**
- Évolution du comportement utilisateur
- Changements saisonniers
- Capteurs défectueux
- Nouvelles sources de données

**Détection:**
- **Tests statistiques** : KS-test, Chi-square, PSI
- **Métriques** : variation de moyenne, variance, histogrammes

**Exemple concret (Diabetes):**
La distribution de la glycémie moyenne augmente fortement après plusieurs mois → le modèle reçoit des profils jamais vus au training.

**Actions:**
- Alerte
- Analyse des features impactées
- Retraining avec données récentes
- Validation avant redéploiement

---

### 2. CONCEPT DRIFT (Dérive du Concept)

**Définition:**
La relation entre les features et la cible change, même si les données semblent similaires.

**Causes:**
- Changements médicaux (nouveaux traitements)
- Évolution des comportements
- Politiques ou règles métier modifiées

**Détection:**
- Baisse progressive des performances
- Erreurs systématiques sur certains segments

**Métriques:**
- Accuracy, F1, AUC, calibration
- Erreur par sous-population

---

### AUTRES MÉTRIQUES IMPORTANTES
- **Latence / Throughput** : temps de réponse et capacité du système
- **Resource Usage** : CPU, RAM, GPU, coûts cloud
- **Business Metrics** : impact réel (revenu, taux d’erreur métier, satisfaction)

---

### DASHBOARD IDÉAL
- Performances du modèle dans le temps
- Data drift par feature
- Concept drift via métriques de sortie
- Latence et ressources système
- Alertes automatiques et historique des versions
